# 变量

## 创建变量

创建变量的最佳方式是调用 tf.get_variable 函数。此函数要求您指定变量的名称。

要使用 tf.get_variable 创建变量，只需提供名称和形状即可

In [1]:
import tensorflow as tf
my_variable = tf.get_variable("my_variable",[1,2,3])

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32,
  initializer=tf.zeros_initializer)

In [3]:
other_variable = tf.get_variable("other_variable", dtype=tf.int32,
  initializer=tf.constant([23, 42]))

## 变量集合

由于 TensorFlow 程序的未连接部分可能需要创建变量，因此能有一种方式访问所有变量有时十分受用。为此，TensorFlow 提供了集合，它们是张量或其他对象（如 tf.Variable 实例）的命名列表。

默认情况下，每个 tf.Variable 都放置在以下两个集合中：

tf.GraphKeys.GLOBAL_VARIABLES - 可以在多台设备间共享的变量，

tf.GraphKeys.TRAINABLE_VARIABLES - TensorFlow 将计算其梯度的变量

如果您不希望变量可训练，可以将其添加到 tf.GraphKeys.LOCAL_VARIABLES 集合中。

In [4]:
my_local = tf.get_variable('my_local',shape=(),
    collections=[tf.GraphKeys.LOCAL_VARIABLES])

或者，您可以将 trainable=False 指定为 tf.get_variable 的参数

In [5]:
my_non_trainable = tf.get_variable('my_non_trainable',shape=(),trainable=False)

您也可以使用自己的集合。集合名称可为任何字符串，且您无需显式创建集合。创建变量（或任何其他对象）后，要将其添加到集合中，请调用 tf.add_to_collection。例如，以下代码将名为 my_local 的现有变量添加到名为 my_collection_name 的集合中：

In [6]:
tf.add_to_collection('my_collection_name',my_local)

要检索您放置在某个集合中的所有变量（或其他对象）的列表，您可以使用：

In [7]:
tf.get_collection('my_collection_name')

[<tf.Variable 'my_local:0' shape=() dtype=float32_ref>]

## 设备放置方式

与任何其他 TensorFlow 指令一样，您可以将变量放置在特定设备上。

在分布式设置中，将变量放置在正确设备上尤为重要。如果不小心将变量放在工作器而不是参数服务器上，可能会严重减慢训练速度，最坏的情况下，可能会让每个工作器不断复制各个变量。为此，我们提供了 tf.train.replica_device_setter，它可以自动将变量放置在参数服务器中。例如：

## 初始化变量

变量必须先初始化后才可使用。如果您在低级别 TensorFlow API 中进行编程（即您在显式创建自己的图和会话），则必须明确初始化变量。tf.contrib.slim、tf.estimator.Estimator 和 Keras 等大多数高级框架在训练模型前会自动为您初始化变量。

请注意，默认情况下，tf.global_variables_initializer 不会指定变量的初始化顺序。因此，如果变量的初始值取决于另一变量的值，那么很有可能会出现错误。任何时候，如果您在并非所有变量都已初始化的上下文中使用某个变量值（例如在初始化某个变量时使用另一变量的值），最好使用 variable.initialized_value()，而非 variable：

In [8]:
m = tf.get_variable("m", shape=(), initializer=tf.zeros_initializer())
w = tf.get_variable("w", initializer=m.initialized_value() + 1)

## 使用变量

In [9]:
v2 = tf.get_variable("v2", shape=(), initializer=tf.zeros_initializer())
w = v2 + 1  # w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value.

要为变量赋值，请使用assign、assign_add方法以及tf.Variable类中的友元

In [10]:
v3 = tf.get_variable("v3", shape=(), initializer=tf.zeros_initializer())
with tf.Session() as sess:
    assignment = v3.assign_add(1)
    tf.global_variables_initializer().run()
    print(sess.run(assignment))  # or assignment.op.run(), or assignment.eval()

1.0


由于变量是可变的，因此及时了解任意时间点所使用的变量值版本有时十分有用。要在事件发生后强制重新读取变量的值，可以使用 tf.Variable.read_value。

In [11]:
v4 = tf.get_variable('v4',shape=(),initializer=tf.zeros_initializer())
assignment1 = v4.assign_add(1)
with tf.control_dependencies([assignment1]):
    w1 =  v4.read_value()

## 共享变量

TensorFlow 支持两种共享变量的方式：

1、显式传递 tf.Variable 对象。

2、在 tf.variable_scope 对象内隐式封装 tf.Variable 对象。

虽然显式传递变量的代码非常清晰，但有时编写在其实现中隐式使用变量的 TensorFlow 函数非常方便。tf.layer 中的大多数功能层以及所有 tf.metrics 和部分其他库实用程序都使用这种方法。

变量作用域允许您在调用隐式创建和使用变量的函数时控制变量重用。作用域还允许您以分层和可理解的方式命名变量。

In [12]:
def conv_relu(input,kernel_shape,bias_shape):
    # Create  variable named "weight".
    weights = tf.get_variable("weight",kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named 'biases'.
    biases = tf.get_variable('biases',bias_shape,
        initializer=tf.constant_initializer())
    conv = tf.nn.conv2d(input,weights,strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(conv + biases)

In [13]:
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1,kernel_shape=[5,5,32,32],bias_shape=[32])
#x = conv_relu(x,kernel_shape=[5,5,32,32],bias_shape=[32])    ## This fails.

由于期望的操作不清楚（创建新变量还是重新使用现有变量？），因此 TensorFlow 将会失败。不过，在不同作用域内调用 conv_relu 可表明我们想要创建新变量：

In [14]:
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        #Variables created here will be named "conv1/weights","conv1/biases".
        relu1 = conv_relu(input_images,[5,5,32,32],[32])
    with tf.variable_scope("conv2"):
        #Variable created here will be named "conv2/weights","conv2/biases".
        return conv_relu(relu1,[5,5,32,32],[32])

如果您想要共享变量，有两种方法可供选择。首先，您可以使用 reuse=True 创建具有相同名称的作用域：

In [15]:
with tf.variable_scope("model"):
    output1 = my_image_filter(input1)
with tf.variable_scope("model",reuse=True):
    output2 = my_image_filter(input2)

您也可以调用 scope.reuse_variables() 以触发重用：

由于依赖于作用域的确切字符串名称可能比较危险，因此也可以根据另一作用域初始化某个变量作用域：